# Fine-tune a BLOOM-based SQL generation model using `peft`, `transformers` and `bitsandbytes`

We can use the [Questions and SQL with DDL Dataset](https://huggingface.co/datasets/stjarvie/question_to_sql_with_ddl) to fine-tune BLOOM to be able to generate simple SQL statements.

### Overview of PEFT and LoRA:

Based on some awesome new research [here](https://github.com/huggingface/peft), we can leverage techniques like PEFT and LoRA to train/fine-tune large models a lot more efficiently. 

It can't be explained much better than the overview given in the above link: 

```
Parameter-Efficient Fine-Tuning (PEFT) methods enable efficient adaptation of
pre-trained language models (PLMs) to various downstream applications without 
fine-tuning all the model's parameters. Fine-tuning large-scale PLMs is often 
prohibitively costly. In this regard, PEFT methods only fine-tune a small 
number of (extra) model parameters, thereby greatly decreasing the 
computational and storage costs. Recent State-of-the-Art PEFT techniques 
achieve performance comparable to that of full fine-tuning.
```

### Install requirements

First, run the cells below to install the requirements:

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (p

### Model loading

Here let's load the `bloom-1b7` model!

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7", 
    load_in_8bit=True, 
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/tokenizer")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


### Post-processing on the model

Finally, we need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast the layer-norm in `float32` for stability. We also cast the output of the last layer in `float32` for the same reasons.

In [3]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Apply LoRA

Here comes the magic with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.

In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:
from peft import LoraConfig, get_peft_model 

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3145728 || all params: 1725554688 || trainable%: 0.18230242262828822


### Preprocessing

We can simply load our dataset from 🤗 Hugging Face with the `load_dataset` method!

In [6]:
import transformers
from datasets import load_dataset

dataset_name = "stjarvie/question_to_sql_with_ddl"
#product_name = "product"
#product_desc = "description"
#product_ad = "ad"

In [7]:
dataset = load_dataset(dataset_name)
print(dataset)
print(dataset['train'][0])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/stjarvie___parquet/stjarvie--question_to_sql_with_ddl-2680149bb0ed4284/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'sql', 'schema'],
        num_rows: 10
    })
})
{'question': 'How many cars are in the database?', 'sql': 'SELECT COUNT(*) FROM cars;', 'schema': 'cars (id INT, make VARCHAR(255), model VARCHAR(255), year INT, color VARCHAR(255))'}


We want to put our data in the form:

```
Below is a product and description, please write an ad for this product.

### Product and Description:
PRODUCT NAME AND DESCRIPTION HERE

### Ad:
OUR AD HERE
```

This way, we can prompt our model well and receive the responses we want!

This is what fine-tuning, and prompt-engineering, is really all about!

In [26]:
def generate_prompt(question: str, schema: str, sql: str) -> str:
  prompt = f"Below is a question and SQL table schema, please write a SQL query for this question using the table schema.\n\n### Question:\n{question}\n\n### Table Schema:\n{schema}\n\n### SQL Query:\n{sql}"
  return prompt

def generate_prompt_inference(question: str, schema: str) -> str:
  prompt = f"### Question:\n{question}\n\n### Table Schema:\n{schema}\n\n### SQL Query:\n "
  return prompt

dataset = dataset.map(lambda samples: tokenizer(generate_prompt(samples['question'], samples['schema'], samples['sql'])))

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [16]:
f
print(generate_prompt(row["question"], row["schema"], row["sql"]))


Below is a question and SQL table schema, please write a SQL query for this question using the table schema.

### Question:
How many cars are in the database?

### Table Schema:
cars (id INT, make VARCHAR(255), model VARCHAR(255), year INT, color VARCHAR(255))

### SQL Query:
SELECT COUNT(*) FROM cars;


In [20]:
trainer = transformers.Trainer(
    model=model, 
    train_dataset=dataset['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        max_steps=100, 
        learning_rate=1e-3, 
        fp16=True,
        logging_steps=1, 
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
with torch.autocast("cuda"):
  trainer.train()

/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,1.825000
2,0.619600
3,1.204400
4,1.204300
5,0.598100
6,1.757400
7,1.741600
8,0.564700
9,1.102800
10,1.053700


## Share adapters on the 🤗 Hub

In [21]:
HUGGING_FACE_USER_NAME = "stjarvie"

In [22]:
from huggingface_hub import notebook_login
notebook_login()

In [23]:
model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/bloom-1b7-sql-generation", use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/stjarvie/bloom-1b7-sql-generation/commit/b62753f6996def8118f2c665eb6e25269ce1e195', commit_message='Upload model', commit_description='', oid='b62753f6996def8118f2c665eb6e25269ce1e195', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

You can also directly load adapters from the Hub using the commands below:

In [25]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = f"{HUGGING_FACE_USER_NAME}/bloom-1b7-sql-generation"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


## Inference

You can then directly use the trained model or the model that you have loaded from the 🤗 Hub for inference as you would do it usually in `transformers`.

### Take it for a spin!

In [38]:
from IPython.display import display, Markdown

def make_inference(question, schema, model_instance):
  batch = tokenizer(generate_prompt_inference(question, schema), return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = model_instance.generate(**batch, max_new_tokens=50)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

In [41]:
sample_question = "Which cars are of make BMW? ?"
sample_schema = "car (id INT, make VARCHAR(255), model VARCHAR(255), month VARCHAR(255), color VARCHAR(255))"

make_inference(sample_question, sample_schema, model)

### Question:
Which cars are of make BMW? ?

### Table Schema:
car (id INT, make VARCHAR(255), model VARCHAR(255), month VARCHAR(255), color VARCHAR(255));

### SQL Query:
SELECT * FROM car WHERE make='BMW';

### Table SQL Schema:
car (id INT, make VARCHAR(255), model VARCHAR(255), year INT, color VARCHAR(255))

### SQL Query:

## Baseline Comparison


In [44]:
prompt = generate_prompt_inference(sample_question, sample_schema)
print(prompt)

### Question:
Which cars are of make BMW? ?

### Table Schema:
car (id INT, make VARCHAR(255), model VARCHAR(255), month VARCHAR(255), color VARCHAR(255))


In [43]:
baseline_model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7", 
    load_in_8bit=True, 
    device_map='auto',
)
batch = tokenizer(generate_prompt_inference(sample_question, sample_schema), return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = baseline_model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.




 ### Question:
Which cars are of make BMW? ?

### Table Schema:
car (id INT, make VARCHAR(255), model VARCHAR(255), month VARCHAR(255), color VARCHAR(255));

### Question:
Which cars are of make BMW? ?

### Table Schema:
car (id INT, make VARCHAR(255), model VARCHAR(255), month VARCHAR(255), color VARCHAR(255));

### Question:



### Example in Training Set

In [37]:

batch = tokenizer(generate_prompt_inference(row["question"], row["schema"]), return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1443: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")




 ### Question:
How many cars are in the database?

### Table Schema:
cars (id INT, make VARCHAR(255), model VARCHAR(255), year INT, color VARCHAR(255));

### SQL Query:
SELECT COUNT(*) FROM cars;

### Result:
3

### Why This Question Is Important:
This question is important because it can be used to calculate the average car's make, model, year


In [48]:
generate_prompt_inference(row["question"], row["schema"])

'### Question:\nHow many cars are in the database?\n\n### Table Schema:\ncars (id INT, make VARCHAR(255), model VARCHAR(255), year INT, color VARCHAR(255))'

In [42]:
### Baseline comparison
baseline_model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7", 
    load_in_8bit=True, 
    device_map='auto',
)
batch = tokenizer(generate_prompt_inference(row["question"], row["schema"]), return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = baseline_model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.




 ### Question:
How many cars are in the database?

### Table Schema:
cars (id INT, make VARCHAR(255), model VARCHAR(255), year INT, color VARCHAR(255));

### Query
SELECT * FROM cars WHERE year = '2015' AND color = 'blue';

### Result
id | make | model | year | color
1  | Ford  | F-150 | 2015 | blue
2  | Ford  |


### Example outside of Training Set

### Example outside of immediate domain

In [47]:
batch = tokenizer(generate_prompt_inference("Select a black car", row["schema"]), return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Question:
Select a black car

### Table Schema:
cars (id INT, make VARCHAR(255), model VARCHAR(255), year INT, color VARCHAR(255))

### SQL Query:
SELECT * FROM cars WHERE color='black';

### Result:
###   id        make        model        year        color
------  ----  ----  ----  ----  ----  ----  ----  ----



In [ ]:
def levenshtein_distance(s, t):
    # Initialize a matrix of zeros with dimensions (len(s)+1) x (len(t)+1)
    d = [[0 for j in range(len(t)+1)] for i in range(len(s)+1)]
    
    # Fill in the first row and column of the matrix
    for i in range(len(s)+1):
        d[i][0] = i
    for j in range(len(t)+1):
        d[0][j] = j
    
    # Fill in the rest of the matrix
    for i in range(1, len(s)+1):
        for j in range(1, len(t)+1):
            if s[i-1] == t[j-1]:
                substitution_cost = 0
            else:
                substitution_cost = 1
            
            d[i][j] = min(d[i-1][j]+1,   # deletion
                          d[i][j-1]+1,   # insertion
                          d[i-1][j-1]+substitution_cost)  # substitution
            
    return d[len(s)][len(t)]

In [54]:
def get_response_query(output_str):
  found_query_line = False
  for line in output_str.split("\n"):
    if found_query_line: 
      return line
    if "query" in line.lower():
        found_query_line = True
        continue

print(get_response_query(tokenizer.decode(output_tokens[0], skip_special_tokens=True)))

SELECT * FROM cars WHERE color='black';


In [58]:
def make_inference_no_formatting(row, modelv):
  batch = tokenizer(generate_prompt_inference(row["question"], row["schema"]), return_tensors='pt')
  with torch.cuda.amp.autocast():
    output_tokens = modelv.generate(**batch, max_new_tokens=50)
  return tokenizer.decode(output_tokens[0], skip_special_tokens=True)



How many cars are in the database?
tuned model response


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1443: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


SELECT COUNT(*) FROM cars;
baseline model response
SELECT * FROM cars WHERE year = '2015' AND color = 'blue';
What is the average year of the cars in the database?
tuned model response
SELECT AVG(year) FROM cars;
baseline model response
SELECT * FROM cars ORDER BY year DESC;
What are the different colors available for the cars?
tuned model response
SELECT DISTINCT color FROM cars;
baseline model response
None
What is the oldest car in the database?
tuned model response
SELECT * FROM cars ORDER BY year ASC LIMIT 1;
baseline model response
SELECT * FROM cars WHERE year = '2015' AND make = 'Volkswagen';
What is the newest car in the database?
tuned model response
SELECT * FROM cars ORDER BY year DESC LIMIT 1;
baseline model response
SELECT * FROM cars WHERE year = '2015' AND model = 'Volkswagen' AND color = 'Red';
How many cars of each color are in the database?
tuned model response
SELECT color, COUNT(*) FROM cars GROUP BY color;
baseline model response
SELECT * FROM cars WHERE color = '

In [73]:

dataset = load_dataset("stjarvie/question_to_sql_with_ddl")
print(dataset)


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/stjarvie___parquet/stjarvie--question_to_sql_with_ddl-286795ce6f3c5f51/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'sql', 'schema'],
        num_rows: 10
    })
    test: Dataset({
        features: ['question', 'sql', 'schema'],
        num_rows: 10
    })
})


In [89]:
train_output = []
for row in dataset['train']:

  question = row["question"]
  tuned_resp = get_response_query(make_inference_no_formatting(row, model))
  baseline_resp = get_response_query(make_inference_no_formatting(row, baseline_model))

  train_output.append([row, tuned_resp, baseline_resp])





/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1443: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


## Train Set Inference Comparison

In [91]:

for (q,t,b) in train_output:
  print("--------------------------------------------------------------------------------------------------- ")
  print(q["question"])

  print(" expected:\t" + q['sql'])
  print(" baseline:\t" + str(b))
  print(" tuned: \t" + str(t))
  print("\n")

--------------------------------------------------------------------------------------------------- 
How many cars are in the database?
 expected:	SELECT COUNT(*) FROM cars;
 baseline:	SELECT * FROM cars WHERE year = '2015' AND color = 'blue';
 tuned: 	SELECT COUNT(*) FROM cars;


--------------------------------------------------------------------------------------------------- 
What is the average year of the cars in the database?
 expected:	SELECT AVG(year) FROM cars;
 baseline:	SELECT * FROM cars ORDER BY year DESC;
 tuned: 	SELECT AVG(year) FROM cars;


--------------------------------------------------------------------------------------------------- 
What are the different colors available for the cars?
 expected:	SELECT DISTINCT color FROM cars;
 baseline:	None
 tuned: 	SELECT DISTINCT color FROM cars;


--------------------------------------------------------------------------------------------------- 
What is the oldest car in the database?
 expected:	SELECT * FROM cars ORDER

## Test Set Inference Comparison

In [ ]:
test_output = []
for row in dataset['train']:

  question = row["question"]
  tuned_resp = get_response_query(make_inference_no_formatting(row, model))
  baseline_resp = get_response_query(make_inference_no_formatting(row, baseline_model))
  test_output.append([row, tuned_resp, baseline_resp])


In [88]:

for (q,t,b) in output:
  print("--------------------------------------------------------------------------------------------------- ")
  print(q["question"])

  print(" expected:\t" + q['sql'])
  print(" baseline:\t" + str(b))
  print(" tuned: \t" + str(t))
  print("\n")

--------------------------------------------------------------------------------------------------- 
What are the unique car makes in the database?
 expected:	SELECT DISTINCT make FROM cars;
 baseline:	SELECT * FROM cars WHERE make = 'Volkswagen' AND model = 'Volkswagen' AND year = '2015'
 tuned: 	SELECT DISTINCT make, model, year FROM cars ORDER BY year DESC LIMIT 1;


--------------------------------------------------------------------------------------------------- 
Which cars are made between the years 2010 and 2020?
 expected:	SELECT * FROM cars WHERE year BETWEEN 2010 AND 2020;
 baseline:	None
 tuned: 	SELECT * FROM cars WHERE year=2020;


--------------------------------------------------------------------------------------------------- 
How many cars are made by each manufacturer?
 expected:	SELECT make, COUNT() FROM cars GROUP BY make;
 baseline:	None
 tuned: 	SELECT COUNT(*) FROM cars;


-----------------------------------------------------------------------------------------